## Imports

In [ ]:
from datasets import load_dataset
from dataclasses import dataclass, field, fields

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    set_seed
)
from trl import SFTTrainer, SFTConfig, TrlParser, setup_chat_format
from peft import LoraConfig

from sklearn.model_selection import train_test_split

import logging
import torch

import os
import json
import random
import numpy as np

## Dataset setting

네이버 지식인 베스트 질문 크롤링: 제목/본문/채택된 답변 본문

In [ ]:
## 원시 데이터 로드
ds = load_dataset("beomi/KoAlpaca-v1.1a")
columns_to_remove = list(ds["train"].features)  ## 전처리 이후 제거할 기존 피쳐

system_prompt = "당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다.\
 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도\
 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인\
 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이\
 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요."

In [ ]:
## 전처리 이전 데이터 예시
ds["train"][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [ ]:
## 전처리 후 저장
train_ds = ds.map(
    lambda sample:
    {"messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["instruction"]},
        {"role": "assistant", "content": sample["output"]}
    ]}
)

train_ds = train_ds.map(remove_columns = columns_to_remove, batched = False)
train_ds = train_ds["train"].train_test_split(test_size = 0.1, seed = 42)

train_ds["train"].to_json("train_dataset.json", orient = "records", force_ascii = False)
train_ds["test"].to_json("test_dataset.json", orient = "records", force_ascii = False)

Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 13.43ba/s]


3811926

In [ ]:
## 전처리 이후 데이터 예시
train_ds["train"][0]

{'messages': [{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
   'role': 'system'},
  {'content': '생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?', 'role': 'user'},
  {'content': '생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. \n\n- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. \n- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. \n- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. \n- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. \n\n위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.',
   'role': 'assistant'}]}

## Model: [Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

* 일단 시험용으로 작은 모델인 3.1-8B instruct를 사용
* nf4 양자화, QLoRA, 
* VRAM 20GB 요구
* huggingface의 trl 라이브러리를 활용

In [ ]:
print(tokenizer.apply_chat_template(train_ds[0]["messages"], tokenize = False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. 

- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. 
- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. 
- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. 
- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. 

위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.<|eot_id|>


`-` 하이퍼 파라미터

```{raw}
## ScriptArguments
model_name: "meta-llama/Meta-Llama-3.1-8B-Instruct"   ## 사용할 모델명 (huggingface model name)
dataset_path: "."                                     ## 데이터셋 저장 경로

## SFTConfig
max_length: 512                                       ## 모델이 수용 가능한 최대 시퀀스 길이
output_dir: "./results/assistant-only-8-epoch-test"   ## 튜닝된 모델 저장 위치
report_to: "wandb"                                    ## 튜닝 로그 리포트 (토큰 등록 필요)
assistant_only_loss: true                             ## LM type dataset: 현재 손실 계산 시 LLM이 답변할 부분만 사용 (user, system messages 무시)
learning_rate: 8e-5                                   ## update matrix 학습률
lr_scheduler_type: "cosine_with_restarts"             ## 코사인 스케줄러 사용 + 학습률 초기화
lr_scheduler_kwargs:
  num_cycles: 5                                       ## 코사인 곡선 사이클 횟수 지정
num_train_epochs: 8                                   ## 에폭
per_device_train_batch_size: 4                        ## GPU당 배치 사이즈
per_device_eval_batch_size: 4                         ## GPU당 배치 사이즈(평가)
gradient_accumulation_steps: 4                        ## 그래디언트를 모아두었다가 한꺼번에 적용: 배치 사이즈를 키우는 효과
do_eval: true
eval_steps: 500
eval_strategy: "steps"
optim: "adamw_torch_fused"                            ## optimizer 설정
logging_steps: 100                                    ## 로그 산출 빈도
save_strategy: "epoch"                                ## 에폭별 모델 저장
weight_decay: 0.01                                    ## l2-norm weight decay. 과적합 방지
max_grad_norm: 0.5                                    ## 그래디언트 클리핑의 임계값 지정. 모든 파라미터의 그래디언트 합 norm의 임계값. exploding 방지
warmup_ratio: 0.06                                    ## 초기 학습률 warmup 단계의 비중 설정: 총 스텝 중 비율
bf16: true                                            ## 
tf32: true                                            ## 
gradient_checkpointing: true                          ## 그래디언트를 캐시에 저장하지 않고 필요할 때마다 계산하여 GPU 절약
packing: true                                         ## 
dataloader_num_workers: 8                             ## 데이터로더 워커 수: 보통 GPU 개수 * 4 정도 사용한다고 함
push_to_hub: true                                     ## 허깅페이스에 모델 로드
dataset_kwargs:
  add_special_tokens: false                           ## 
  append_concat_token: false                          ## 

## LoraArguments
r: 64                                                 ## update matrix의 rank. 작을수록 많이 압축하여 품질 저하됨, 메모리 많이 할당됨
lora_alpha: 32                                        ## ∆Weight scaling factor. lora_alpha / r로 스케일링되며, 학습률 조정. 보통 1/2 수준으로 설정
lora_dropout: 0.05                                    ## update matrics에서 dropout 적용 확률
bias: "none"                                          ## update matrix에 bias를 학습할 것인지 선택
task_type: "CAUSAL_LM"                                ## 튜닝 모형의 유형 지정
```

## 모델 테스트: VRAM 16GB 요구

In [ ]:
import os
import torch
from random import randint
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`-` 테스트에 사용될 데이터

In [ ]:
test_dataset = load_dataset("json", data_files = os.path.join("", "./test_dataset.json"), split = "train")
random_idx = randint(0, len(test_dataset))
messages = test_dataset[random_idx]["messages"][:2]

In [ ]:
messages

[{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
  'role': 'system'},
 {'content': '미세먼지가 나쁨일 때, 어떻게 환기를 해야 할까요?\n미세먼지가 나쁨일 때에도 깨끗한 공기를 유지하려면 어떻게 해야 할까요?',
  'role': 'user'}]

`-` 기존 모델 결과(파인튜닝 이전)

In [ ]:
origin_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

origin_model = AutoModelForCausalLM.from_pretrained(origin_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
origin_tokenizer = AutoTokenizer.from_pretrained(origin_model_name, use_fast = True)
origin_tokenizer.pad_token = origin_tokenizer.eos_token
origin_tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


In [ ]:
terminators = [origin_tokenizer.eos_token_id]

input_ids = origin_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(origin_model.device)

outputs = origin_model.generate(
    input_ids,
    max_new_tokens = 512,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_idx]["messages"][1]["content"]}\n")
print(f"원답변:\n{test_dataset[random_idx]["messages"][2]["content"]}\n")
print(f"생성답변:\n{origin_tokenizer.decode(response, skip_special_tokens = True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


질문:
미세먼지가 나쁨일 때, 어떻게 환기를 해야 할까요?
미세먼지가 나쁨일 때에도 깨끗한 공기를 유지하려면 어떻게 해야 할까요?

원답변:
미세먼지가 나쁨일 때에는 창문을 여는 것이 어렵습니다. 하지만 출퇴근 시간과 같이 차량이 많은 시간을 피하고, 일일이 창문을 활짝 열어 환기하는 것보다는 최단시간(3분 이내) 환기를 해야합니다. 창문을 열 때, 써큘레이터나 선풍기를 이용해 공기 흐름을 도와주시는 것이 좋습니다. 추가로, 환기 후에는 분무기를 활용해 공기 중의 미세먼지가 바닥에 떨어진 뒤 물걸레로 집안을 청소해주세요. 하지만 이것이 모든 미세먼지를 제거해주는 것은 아닙니다. 따라서 공기 청정기를 이용하시는 것도 좋은 방법입니다. 그리고 미세먼지 예방에 도움이 되는 것으로 창문에 방충망과 공기 청정 필터를 설치하는 것도 좋은 방법 중 하나입니다.  위의 방법과 함께 유의미한 효과를 볼 수 있을 것입니다.

생성답변:
미세먼지가 나쁨일 때는 환기를 잘 하여 깨끗한 공기를 유지하는 것이 중요합니다. 다음과 같이 환기를 잘 하실 수 있습니다.

1.  **창문을 열고 환기**: 미세먼지가 나쁨일 때 창문을 열고 환기를 하세요. 환기하는 동안 창문을 닫지 마세요. 미세먼지는 공기가 움직일 때 더 쉽게 새어나옵니다.
2.  **공기청정기 사용**: 공기청정기를 사용하여 미세먼지를 잡아주세요. 공기청정기는 미세먼지를 걸러내어 깨끗한 공기를 만들 수 있습니다.
3.  **습도 조절**: 습도가 높을 때 미세먼지가 더 잘 퍼집니다. 공기를 마른 상태에서 환기를 하세요. 에어컨이나 히터를 사용하여 공기를 마르게 하세요.
4.  **미세먼지 측정**: 미세먼지를 측정하여 환기할 때 어떤 상태인지 확인하세요. 미세먼지 측정기는 미세먼지 농도를 측정하여 환기할 때 어떤 상태인지 알려줍니다.
5.  **환기 시간**: 환기 시간을 정해두고 유지하세요. 미세먼지가 나쁨일 때는 매일 환기를 하세요. 환기 시간을 정해두면 미세먼지를 잡아주는 데 도움이 됩니다.
6.  **공기청정기 필터

In [ ]:
del origin_model
del origin_tokenizer
torch.cuda.empty_cache()

`-` 파인튜닝 후 모델

In [ ]:
SFT_model_name = "./results/llama-3.1-korean-8b-4-epoch/checkpoint-4752"

SFT_model = AutoModelForCausalLM.from_pretrained(SFT_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
SFT_tokenizer = AutoTokenizer.from_pretrained(SFT_model_name, use_fast = True)
SFT_tokenizer.pad_token = SFT_tokenizer.eos_token
SFT_tokenizer.pad_token_id = SFT_tokenizer.eos_token_id
SFT_tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.25s/it]


In [ ]:
terminators = [SFT_tokenizer.eos_token_id]

input_ids = SFT_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(SFT_model.device)

outputs = SFT_model.generate(
    input_ids,
    max_new_tokens = 512,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_idx]["messages"][1]["content"]}")
print(f"원답변:\n{test_dataset[random_idx]["messages"][2]["content"]}")
print(f"생성답변:\n{SFT_tokenizer.decode(response, skip_special_tokens = True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
미세먼지가 나쁨일 때, 어떻게 환기를 해야 할까요?
미세먼지가 나쁨일 때에도 깨끗한 공기를 유지하려면 어떻게 해야 할까요?
원답변:
미세먼지가 나쁨일 때에는 창문을 여는 것이 어렵습니다. 하지만 출퇴근 시간과 같이 차량이 많은 시간을 피하고, 일일이 창문을 활짝 열어 환기하는 것보다는 최단시간(3분 이내) 환기를 해야합니다. 창문을 열 때, 써큘레이터나 선풍기를 이용해 공기 흐름을 도와주시는 것이 좋습니다. 추가로, 환기 후에는 분무기를 활용해 공기 중의 미세먼지가 바닥에 떨어진 뒤 물걸레로 집안을 청소해주세요. 하지만 이것이 모든 미세먼지를 제거해주는 것은 아닙니다. 따라서 공기 청정기를 이용하시는 것도 좋은 방법입니다. 그리고 미세먼지 예방에 도움이 되는 것으로 창문에 방충망과 공기 청정 필터를 설치하는 것도 좋은 방법 중 하나입니다.  위의 방법과 함께 유의미한 효과를 볼 수 있을 것입니다.
생성답변:
미세먼지가 나쁨일 때, 환기를 위해 다음을 참고해주세요.

1. **지하실을 이용하여 환기** : 지하실에서 환기를 하거나, 창문을 열어 환기를 합니다.
2. **가스레인지 사용을 자제** : 가스레인지 사용은 미세먼지 발생을 증가시켜 환기를 어렵게 만들 수 있습니다. 전기레인지 사용이 더 좋습니다.
3. **전기 가전제품 사용을 줄이기** : 전기 가전제품 사용을 최소화하는 것이 좋습니다. 특히, 미세먼지 발생을 증가시킬 수 있는 제품은 사용을 자제해야 합니다.
4. **공기 청정기 사용** : 공기 청정기를 사용하여 미세먼지를 제거하는 것이 좋습니다. 공기 청정기는 미세먼지와 함께 많은 양의 불순물이 있는 공기를 깨끗한 공기로 변환시켜 주는 제품입니다.
5. **천장 위에 창문을 열어 환기** : 천장 위에 있는 창문을 열어 환기를 합니다. 천장은 미세먼지가 가라앉지 않으므로, 창문을 열어 환기를 해야 합니다.
6. **기러기처럼 창문을 열어 환기** : 기러기처럼 창문을 열어 환기를 합니다. 창문을 열어 환기를 하면, 미세먼지가 빠르게 제거될 